# DCTによる圧縮画像を作る

In [7]:
import numpy as np
import scipy.misc
from scipy.fftpack import dct, idct
import sys
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import random
from tqdm import tqdm_notebook as tqdm
from scipy.fftpack import dct, idct
import seaborn as sns
from skimage.metrics import structural_similarity as ssim
import pandas as pd

%matplotlib inline


class ImageLoader:
    def __init__(self, FILE_PATH):
        self.img = np.array(Image.open(FILE_PATH))
        # 行数
        self.row_blocks_count = self.img.shape[0] // 8
        # 列数
        self.col_blocks_count = self.img.shape[1] // 8

    def get_points(self, POINT):
        Row = random.randint(0, len(self.img) - POINT - 1)
        Col = random.randint(0, len(self.img) - 1)
        return self.img[Row : Row + POINT, Col]

    def get_block(self, col, row):
        return self.img[col * 8 : (col + 1) * 8, row * 8 : (row + 1) * 8]


# plt.rcParams['font.family'] ='sans-serif'#使用するフォント
# plt.rcParams["font.sans-serif"] = "Source Han Sans"
plt.rcParams["font.family"] = "Source Han Sans JP"  # 使用するフォント
plt.rcParams["xtick.direction"] = "in"  # x軸の目盛線が内向き('in')か外向き('out')か双方向か('inout')
plt.rcParams["ytick.direction"] = "in"  # y軸の目盛線が内向き('in')か外向き('out')か双方向か('inout')
plt.rcParams["xtick.major.width"] = 1.0  # x軸主目盛り線の線幅
plt.rcParams["ytick.major.width"] = 1.0  # y軸主目盛り線の線幅
plt.rcParams["font.size"] = 12  # フォントの大きさ
plt.rcParams["axes.linewidth"] = 1.0  # 軸の線幅edge linewidth。囲みの太さ
matplotlib.font_manager._rebuild()

# 画像の用意
MONO_DIR_PATH = "../../Mono/"
OUTPUT_DIR_PATH = "../../Plot_tool/output/images/DCT/32x32/"

AIRPLANE = ImageLoader(MONO_DIR_PATH + "airplane512.bmp")
BARBARA = ImageLoader(MONO_DIR_PATH + "barbara512.bmp")
BOAT = ImageLoader(MONO_DIR_PATH + "boat512.bmp")
GOLDHILL = ImageLoader(MONO_DIR_PATH + "goldhill512.bmp")
LENNA = ImageLoader(MONO_DIR_PATH + "lenna512.bmp")
MANDRILL = ImageLoader(MONO_DIR_PATH + "mandrill512.bmp")
MILKDROP = ImageLoader(MONO_DIR_PATH + "milkdrop512.bmp")
SAILBOAT = ImageLoader(MONO_DIR_PATH + "sailboat512.bmp")
TEST02 = ImageLoader(MONO_DIR_PATH + "test02.bmp")
TEST19 = ImageLoader(MONO_DIR_PATH + "test19.bmp")

Images = np.array(
    [
        AIRPLANE,
        BARBARA,
        BOAT,
        GOLDHILL,
        LENNA,
        MANDRILL,
        MILKDROP,
        SAILBOAT
    ]
)

Images = np.array([
    TEST02,TEST19
])

N = 32

# 量子化テーブル

In [8]:

Q50_Luminance = np.array(
    [
        [16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12, 14, 19, 26, 58, 60, 55],
        [14, 13, 16, 24, 40, 57, 69, 56],
        [14, 17, 22, 29, 51, 87, 80, 62],
        [18, 22, 37, 56, 68, 109, 103, 77],
        [24, 35, 55, 64, 81, 104, 113, 92],
        [49, 64, 78, 87, 103, 121, 120, 101],
        [72, 92, 95, 98, 112, 100, 103, 99],
    ]
)

# 8x8ブロックでDCT

In [9]:
option = str(N) + "x" + str(N)
img_name = ["Airplane","Barbara","Boat","Goldhill","Lenna","Mandrill","Milkdrop","Sailboat"]
img_name = ["TEST02","TEST19"]
df = pd.read_excel("../../Plot_tool/xlsx/DCT_"+ option +".xlsx",sheet_name=None)

for i in range(len(Images)):
    name = img_name[i]
    IMG = Images[i]
    df_Q = df[name]["Q"]
 
    for j in range(13):
        bpp = 0.2 + 0.05 * j
        Q = df_Q[j]
        
        Fk = np.zeros(IMG.img.shape)
        for row in range(IMG.img.shape[0] // N):
            for col in range(IMG.img.shape[1] // N):
                block = IMG.img[row * N : (row + 1) * N, col * N : (col + 1) * N]
                c = scipy.fftpack.dct(
                    scipy.fftpack.dct(block, axis=0, norm="ortho"), axis=1, norm="ortho"
                )
                Fk[row * N : (row + 1) * N, col * N : (col + 1) * N] = c
        
#         Q_Luminance = np.copy(Q50_Luminance)
#         if Q < 50:
#             Q_Luminance = Q_Luminance * 50 / Q
#             Q_Luminance = np.where(Q_Luminance > 255, 255, Q_Luminance)
#         else:
#             Q_Luminance = (100 - Q) * Q50_Luminance / 50
#             Q_Luminance = np.where(Q_Luminance < 1, 1, Q_Luminance)
        
        Q_Luminance = np.ones((N,N)) * Q
        
        Q_Fk = np.zeros(Fk.shape)
        for row in range(IMG.img.shape[0] // N):
            for col in range(IMG.img.shape[1] // N):
                block = Fk[row * N : (row + 1) * N, col * N : (col + 1) * N]
                # 量子化
                block = np.round(block / Q_Luminance)
                # 逆量子化
                block = block * Q_Luminance
                Q_Fk[row * N : (row + 1) * N, col * N : (col + 1) * N] = block
                
        Q_recover = np.zeros(Q_Fk.shape)
        for row in range(Q_Fk.shape[0] // N):
            for col in range(Q_Fk.shape[1] // N):
                block = Q_Fk[row * N : (row + 1) * N, col * N : (col + 1) * N]
                data = scipy.fftpack.idct(
                    scipy.fftpack.idct(block, axis=0, norm="ortho"), axis=1, norm="ortho"
                )
                Q_recover[row * N : (row + 1) * N, col * N : (col + 1) * N] = data
        
        Q_recover = np.round(Q_recover)
        plt.imsave(OUTPUT_DIR_PATH + "DCT_" + option + "_" + name + "_" + "{:.2f}".format(bpp) + ".png",Q_recover,cmap="gray")
#         pil_img_gray = Image.fromarray(np.uint8(Q_recover))
#         pil_img_gray.save(OUTPUT_DIR_PATH + "DCT_" + option + "_" + name + "_" + "{:.2f}".format(bpp) + ".jpg")